In [ ]:
# Import Libraries
import os.path
import numpy as np
from tensorflow.python.platform import gfile
import pypcd
import os
import subprocess
import pandas as pd
from pandas import DataFrame

In [ ]:
# Find number of PointClouds
point_dir = '/home/johan/Documents/Alignment/Points_csv/'
#se crea un vector vacio para guardar los nombres de las imágenes.
file_list = []
#se obtienen todos los nombre de las imagenes en la carpeta train que tengan extensión .csv
file_glob = os.path.join(point_dir, '*.csv')
#se organizan en el vector file_list todos las direcciones de las imágenes 
#encontrados dentro de la carpeta train
file_list.extend(gfile.Glob(file_glob))
file_list = np.sort(file_list)
#kind : {‘quicksort’, ‘mergesort’, ‘C’, ‘stable’}
print(len(file_list))
print(type(file_list[0]),file_list[0:8])

In [ ]:
# Alignt PCs
#para cada pointcloud dentro del vector file_list
a = 0 
refetotal = []
for point_dir in file_list:
    if a == 0:
        refe = pd.read_csv(point_dir)
        df = DataFrame(refe, columns= ['x', 'y','z'])
        export_csv = df.to_csv ('/home/johan/Documents/Alignment/Points_csv/refe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path
        #print ('a0:\n refe',df.shape)
        #print('ICP finished succesfully')
        #print('------------------------------------------------------')
        
    else:
        pointcloud1_path_refe = "/home/johan/Documents/Alignment/Points_csv/refe.csv"
        refe = pd.read_csv(pointcloud1_path_refe)
        refe = np.array(refe)
        #print ("refe", refe.shape) 
        
        pointcloud2_path_in = point_dir
        icp_path="/home/johan/Libraries/libpointmatcher/build/examples"

        #--------------- correr el ejecutable de c++ ------------------
        #runicp = subprocess.Popen([os.path.join(icp_path,"icp_simple"),pointcloud1_path,pointcloud2_path],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        runicp = subprocess.Popen([os.path.join(icp_path,"icp_simple"),pointcloud1_path_refe,pointcloud2_path_in,'OutPc'],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        runicp.wait()
        #-------------leer los datos de devuelve en el terminal------
        #stdout, stderr = runicp.communicate()
        
        #------------------------------------------------------------
        refe2 = pd.read_csv('/home/johan/repos/GitHub/3D-Reconstruction/refe_OutPc.csv')
        refetotal = DataFrame(refe2, columns= ['x','y','z'])
        refetotal = np.array(refetotal)
        #print('refe_OutPc',refetotal.shape, type(refetotal))
        refetotal = refetotal[1:refetotal.shape[0],:]
        #print('refe_OutPc-WOut[x,y,z]',refetotal.shape)
        refetotal_l = np.vstack((refe,refetotal))
        refetotal = DataFrame(refetotal_l) 
        #print('Acumulado',refetotal.shape)
        export_csv = refetotal.to_csv ('/home/johan/Documents/Alignment/Points_csv/refe.csv', index = None, header=True) 
        #print('ICP finished succesfully')
        #print('------------------------------------------------------') 
    
    a = a +1
    print('a:',a)

## Running ICP

In [1]:
import numpy as np
#from open3d import *
import pypcd
import os
import subprocess

In [41]:
if __name__ == '__main__':

    #------- rutas hacia las nubes de puntos y el ejecutable----------
    # .cpp file: addres /home/johan/Libraries/libpointmatcher/examples
    icp_path="/home/johan/Libraries/libpointmatcher/build/examples"
    pointcloud1_path="/home/johan/repos/GitHub/3D-Reconstruction/Dataset_csv/pointscsv1.csv"
    pointcloud2_path="/home/johan/repos/GitHub/3D-Reconstruction/Dataset_csv/pointscsv137.csv"

    #--------------- correr el ejecutable de c++ ------------------
    #runicp = subprocess.Popen([os.path.join(icp_path,"icp_simple"),pointcloud1_path,pointcloud2_path],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    runicp = subprocess.Popen([os.path.join(icp_path,"icp_simple"),pointcloud1_path,pointcloud2_path,'Out'],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    runicp.wait()
    
    stdout, stderr = runicp.communicate()
    print(stdout,'\n')
    
    # bytes to str, alternative
    a = np.matrix(str(stdout, encoding='ascii'))
    a = a.reshape(4,4)
    print(a, a.shape)

b'   0.996618   0.0821525 -0.00170916     0.97926\n -0.0821504    0.996619  0.00122216 -0.00706542\n 0.00180378 -0.00107762    0.999998  0.00148535\n          0           0           0           1\n' 

[[ 0.996618    0.0821525  -0.00170916  0.97926   ]
 [-0.0821504   0.996619    0.00122216 -0.00706542]
 [ 0.00180378 -0.00107762  0.999998    0.00148535]
 [ 0.          0.          0.          1.        ]] (4, 4)


## Final Code ICP-ROS

In [2]:
import rospy
import message_filters
from sensor_msgs.msg import Image, PointCloud2
import sys
from cv_bridge import CvBridge, CvBridgeError
#import cv2
import numpy as np
import laser_geometry.laser_geometry as lg
import sensor_msgs.point_cloud2 as pc2
import open3d as opn3

In [ ]:
class synchronizer:
	def __init__(self):
		#self.laserProj = LaserProjection()
		self.bridge = CvBridge()
		#self.image_sub = message_filters.Subscriber('/left/image_rect_color', Image)
        self.pc_pub = message_filters.Publisher('/Dense',PointCloud2,queue_size=2)
		self.pc_sub = message_filters.Subscriber('/kitti/velo/pointcloud', PointCloud2,self.Callback)
		
	def callback(self, pointcloud):
         cloud_pub.publish(pointcloud)
        
		
def main(args):
  print('Starting...')
  rospy.init_node('sync_node', anonymous=True)
  rospy.loginfo("sync_node on")
  #sc = synchronizer()
  
  try:
    rospy.spin()
  except KeyboardInterrupt:
    print("Shutting down")

if __name__ == '__main__':
	
    main(sys.argv)